In [1]:
from probemapper.registration import ANTSPointTransformation, ANTSTransformationStages
from probemapper.io import save_nifti
import os, subprocess
import numpy as np
import pandas as pd

In [2]:
rootdir = "/bucket/ReiterU/DBS/data/20220518_P186_laqueus_DiI_L-A2/"
resultdir = os.path.join(rootdir, "analysis")
if not os.path.exists(resultdir):
    subprocess.run(["ssh", "deigo", "mkdir", resultdir])

In [3]:
cwd = os.path.basename(os.getcwd())
tmpdir = os.path.join("/flash/ReiterU/tmp/", cwd, f"indexing")
if not os.path.exists(tmpdir):
    os.makedirs(tmpdir)

### Transform channel coordinates

In [4]:
T = ANTSPointTransformation(
    ants_path="/apps/unit/ReiterU/ANTs/2.3.5/bin/",
    num_threads=8
)
T.set_transformations([
    ANTSTransformationStages(transformation=os.path.join(rootdir, "analysis/ants/F2M_0GenericAffine.mat"), inverse=True),
    ANTSTransformationStages(transformation=os.path.join(rootdir, "analysis/ants/F2M_1InverseWarp.nii.gz"), inverse=False),
])
T.set_moving_points(os.path.join(rootdir, "analysis/skeleton/channel_skeleton.csv"))
T.set_outname(os.path.join(tmpdir, "channel_LUT.csv"))

In [5]:
T.run()

['/apps/unit/ReiterU/ANTs/2.3.5/bin/antsApplyTransformsToPoints', '-d', '3', '-i', '/bucket/ReiterU/DBS/data/20220518_P186_laqueus_DiI_L-A2/analysis/skeleton/channel_skeleton.csv', '-o', '/flash/ReiterU/tmp/P186/indexing/channel_LUT.csv', '-t', '[/bucket/ReiterU/DBS/data/20220518_P186_laqueus_DiI_L-A2/analysis/ants/F2M_0GenericAffine.mat,1]', '-t', '/bucket/ReiterU/DBS/data/20220518_P186_laqueus_DiI_L-A2/analysis/ants/F2M_1InverseWarp.nii.gz']



### Query region ID

In [6]:
from probemapper import skeletonization

In [7]:
df = pd.read_csv(os.path.join(tmpdir, "channel_LUT.csv"))
coords = np.array([df["Z"], df["Y"], df["X"]])

In [8]:
regions = skeletonization.query_region_id2(coords,
                     "/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/Slicer_3D/Segmentation.seg.nrrd",
                     100, coords.shape[1]-100, 10)

In [9]:
df["Region"] = regions

In [10]:
df.insert(0, "channel", df.pop("channel"))

In [11]:
display(df)

,channel,X,Y,Z,X_org,Y_org,Z_org,Region
0,-100,2444.38,3619.72,2796.56,1947.48,2596.60,1070.73,9
1,-99,2443.16,3611.90,2799.35,1945.37,2593.19,1079.97,9
2,-98,2441.99,3603.56,2802.33,1943.25,2589.78,1089.21,9
3,-97,2440.91,3594.70,2805.69,1941.14,2586.38,1098.45,9
4,-96,2439.97,3585.56,2809.37,1939.03,2582.97,1107.68,9
...,...,...,...,...,...,...,...,...
287,187,2470.96,1418.89,4084.02,1313.21,1702.56,3717.83,1
288,188,2471.35,1408.60,4085.29,1310.98,1697.34,3727.12,1
289,189,2471.99,1397.73,4087.07,1308.77,1692.16,3736.41,1
290,190,2473.02,1385.39,4090.02,1306.59,1687.04,3745.71,1


In [12]:
df.to_csv(os.path.join(tmpdir, "channel_LUT.csv"), index=False)

In [13]:
sk = skeletonization.generate_skeleton_image(coords[:, 100:],
                             "/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/Slicer_3D/Segmentation.seg.nrrd",
                             10)

In [14]:
sk = sk.astype("uint8")
save_nifti(sk, os.path.join(tmpdir, "probe_image.nii.gz"), 1, 1, 1)

In [15]:
subprocess.run(["scp", "-r", tmpdir, f"deigo:{resultdir}"])

CompletedProcess(args=['scp', '-r', '/flash/ReiterU/tmp/P186/indexing', 'deigo:/bucket/ReiterU/DBS/data/20220518_P186_laqueus_DiI_L-A2/analysis'], returncode=0)